In [1]:
import os

#Change working directory to /Users/albertvong/Documents/Argonne/PtychoPINN/
os.chdir('/home/av_linux/PtychoPINN')

In [3]:
import numpy as np
import importlib

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Subset

import ptycho.tf_helper as tf_hh
import ptycho_torch.helper as pt_hh


2024-10-14 11:06:09.985365: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-14 11:06:09.985426: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-14 11:06:10.031170: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-14 11:06:10.125071: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-14 11:06:11.380498: W tensorflow/compiler/tf2

No GPU found, using CPU instead.
1 items had no tests:
    __main__
0 tests in 1 items.
0 passed and 0 failed.
Test passed.


/root/miniconda3/envs/ptychopinn_ptychodus_pytorch/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


# We'll be using a random set of tensors to check if the functionalities of the forward model for both tensorflow and pytorch are consistent.

## All tensorflow functions without assertions

In [4]:
import tensorflow as tf

In [5]:
gridsize = 2

In [6]:
def extract_patches_position(imgs: tf.Tensor, offsets_xy: tf.Tensor, jitter: float = 0.) -> tf.Tensor:
    """
    Expects offsets_xy in channel format.

    imgs must be in flat format with a single image per solution region, i.e.
    (batch size, M, M, 1) where M = N + some padding size.

    Returns shifted images in channel format, cropped symmetrically

    no negative sign
    """
    if  imgs.get_shape()[0] is not None:
        assert int(imgs.get_shape()[0]) == int(offsets_xy.get_shape()[0])
    assert int(imgs.get_shape()[3]) == 1
    assert int(offsets_xy.get_shape()[2]) == 2
    assert int(imgs.get_shape()[3]) == 1
    gridsize = gridsize
    assert int(offsets_xy.get_shape()[3]) == gridsize**2
    offsets_flat = tf_hh.flatten_offsets(offsets_xy)
    stacked = tf.repeat(imgs, gridsize**2, axis = 3)
    flat_padded = tf_hh._channel_to_flat(stacked)
    channels_translated = tf_hh.trim_reconstruction(
        tf_hh.Translation()([flat_padded, offsets_flat, jitter]))
    return channels_translated

## All pytorch setup

In [10]:
from ptycho_torch.config_params import TrainingConfig, DataConfig, ModelConfig

In [11]:
data_config_default = {
    'nphotons': 1e5,
    'N': 128,
    'C': 4,
    'K': 6,
    'n_subsample': 10,
    'grid_size': (2,2),
    'probe_dir_get': True,
}


model_config_default = {
    'intensity_scale_trainable': True,
    'max_position_jitter': 10, #Random jitter for translation (helps make model more robust)
    'n_filters_scale': 2, #Shrinking factor for channels
    'intensity_scale': 15000.0, #General intensity scale guess, this can be trainable. Needs to be float
    'object.big': True, #True if need patch reassembly
    'probe.big': True, #True if need patch reassembly
    'offset': 4,
    'loss_function': 'Poisson'
}

training_config_default = {
    'nll': True, #Negative log likelihood for loss function
    'device': 'cuda'
}

t_config, d_config, m_config = TrainingConfig(), DataConfig(), ModelConfig()

t_config.set_settings(training_config_default)
d_config.set_settings(data_config_default)
m_config.set_settings(model_config_default)

# Setting fixed matrices to pass through forward pass

Assume that we have just passed through the autoencoder, which produces a random stack of solution regions with amplitude and phase.

In [8]:
#Fix rng seed
np.random.seed(42)

#Random valued image tensors
x_amp = np.random.randn(4, 4, 128, 128)
x_phi = np.random.randn(4, 4, 128, 128)

#Random valued positions
pos = np.random.uniform(low=-20, high = 20,
                        size = (4, 4, 1, 2))


In [ ]:
#First test with pytorch

#Convert x_amp from numpy to pytorch array

x_amp_pt = torch.from_numpy(x_amp)
x_phi_pt = torch.from_numpy(x_phi)

x_combined_pytorch = pt_hh.combine_complex()
reassembled_patch_pytorch = pt_hh.reassemble_patches_position_real()

